깔아야 하는 라이브러리
[ollama pandas tqdm konlpy scikit-learn numpy hanja kiwipiepy]

In [10]:
import pandas as pd
import numpy as np
import re
import html
import hanja
from datetime import date, timedelta
import os
from kiwipiepy import Kiwi

In [11]:
news_file_name = 'crawled_news_20251005_174551.xlsx'
df_news = pd.read_excel(news_file_name)
df_news=df_news.loc[:,['crawled_news_id','title','pub_date','text']]

FileNotFoundError: [Errno 2] No such file or directory: 'crawled_news_20251005_174551.xlsx'

In [8]:
print(df_news.shape)
df_news.head()

NameError: name 'df_news' is not defined

In [4]:
today = date.today()
# 이건 임의로 어제 기사가 없으므로(나중에 실사용 할때에는 지우기!!!)
today = date(2025, 10, 1)

yesterday = today - timedelta(days=1)

# 만약에 전체 기사로 결과를 보고 싶다면 아래 코드 한줄을 주석 처리
#df_news = df_news[df_news['pub_date'].dt.date == yesterday]
print(yesterday,"의 뉴스 개수 : ",df_news.shape[0])
df_news.head()

NameError: name 'df_news' is not defined

In [21]:
df_news_dup = df_news.copy()

# 중복제거
kr_news = df_news_dup.drop_duplicates(subset=['text'], keep='first').reset_index(drop=True)
removed_duplicates = df_news_dup[df_news_dup.duplicated(subset=['text'], keep='first')].copy()
removed_duplicates["reason"] = "중복"

print("중복 제거로 삭제된 행 : ", removed_duplicates.shape[0])

중복 제거로 삭제된 행 :  175


In [22]:
removed_nan = kr_news[kr_news.isnull().any(axis=1)].copy()
removed_nan["reason"] = "nan값"

# nan값 제거 실행
kr_news = kr_news.dropna()


print("nan값 제거로 삭제된 행 : ",removed_nan.shape[0])

nan값 제거로 삭제된 행 :  0


In [23]:
# 제거할 문구 목록
remove_patterns = [
    '해당기사를 찾을 수 없습니다',
    '인터넷 법률신문은 인터넷신문윤리강령 및 그 실천요강을 준수합니다',
    '국가정보자원관리원 화재로 인하여 현재 임시로 보도자료만 제공됩니다',
    'BEST 댓글 답글과 추천수를 합산하여 자동으로 노출됩니다',
    'SNS 기사보내기 카카오스토리(으)로 기사보내기',
    'SNS 기사보내기 카카오스토리로 기사보내기',
    '슬로우레터는 뉴스를 더 열심히 읽고 구조와 맥락을 이해하기 위한 프로젝트입니다'
]


# 문구가 포함된 행 찾기
mask = kr_news['text'].apply(lambda x: any(pat in str(x) for pat in remove_patterns))
removed_pattern = kr_news[mask].copy()
removed_pattern["reason"] = "잘못된 본문"
kr_news = kr_news[~mask]

print("잘못된 본문이 삭제된 행 개수: ",removed_pattern.shape[0])

잘못된 본문이 삭제된 행 개수:  9


In [24]:
# 제목 정제
kr_news['title'] = kr_news['title'].apply(lambda x: html.unescape(str(x)))
kr_news['title'] = kr_news['title'].apply(lambda x: re.sub(r'[^A-Za-z0-9가-힇.,\"\'\:\·\!\?\-\%\~\&]', ' ', x))
kr_news['title'] = kr_news['title'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

In [25]:
#kiwi라는 형태소 분석 (제목 분석할 때 0kt보다 더 잘됨)
kiwi = Kiwi()

# 제목에서 명사 찾기
def check_noun_title(row, kiwi_analyzer, threshold=0.5):
    title = str(row['title'])
    text = str(row['text'])

    result = kiwi_analyzer.analyze(title)
    
    if not result:
        return (True, [], 1.0, [])
    
    # 제목을 (단어, 품사)로 쪼개기
    tagged_words = result[0][0]
    
    # 명사(NNG), 고유명사(NNP), SL(외국어, 알파벳)만 남김
    title_nouns = [word for word, tag, _, _ in tagged_words 
                   if tag in ['NNG', 'NNP', 'SL']]
    
    if not title_nouns:
        return (True, [], 1.0, tagged_words)

    # 그 명사가 본문에 몇개 들어있는지 확인    
    match_count = 0
    for noun in title_nouns:
        if noun in text:
            match_count += 1
            
    # 제목에서 뽑아온 명사들이 본문에 어떤 비율로 들어있는지
    match_ratio = match_count / len(title_nouns)
    
    return (match_ratio >= threshold, title_nouns, match_ratio, tagged_words)


if kiwi: 
    current_threshold = 0.3

    results_series = kr_news.apply(
        check_noun_title,
        axis=1,
        kiwi_analyzer=kiwi, 
        threshold=current_threshold
    )
    # 기사 매칭이 0.3을 기준으로 true인지 false인지
    mask = results_series.str[0]
    
    removed_mismatch = kr_news[~mask].copy()
    removed_mismatch["reason"] = "제목, 본문 불일치"
    kr_news = kr_news[mask]

    print("제목-본문 매칭이 되지 않아 삭제된 행 개수 : ", removed_mismatch.shape[0])

제목-본문 매칭이 되지 않아 삭제된 행 개수 :  74


In [26]:
# 본문 정제
# HTML 이스케이프 해제 및 한자 변환
kr_news['text'] = kr_news['text'].apply(lambda x: html.unescape(str(x)))
kr_news['text'] = kr_news['text'].apply(lambda x: hanja.translate(x, 'substitution'))

# 제거할 패턴들
patterns_to_remove = [
    r'\([^)]*\)|\[[^\]]*\]|\{[^}]*\}|<[^>]*>',
    r'\S*@\S*',
    r'http\S+|www\S+',
    r'\S*=\S*',
    r'[가-힣]{2,4}\s?(기자|특파원)',
    r'(연합뉴스|뉴스1|뉴시스|조선일보|중앙일보|동아일보|한겨레|한국일보|서울경제|매일경제|머니투데이|한국경제|경향신문|헤럴드경제|아시아경제|이데일리|데일리안|세계일보|국민일보|뉴스핌|파이낸셜뉴스)',
    r'(무단전재\s*및\s*재배포\s*금지|저작권자[^.,\n]+|Copyright\s*ⓒ[^.,\n]+|끝\)|끝$)',
    r'(사진\s*=\s*[^.,\n]+|관련기사|주요뉴스|이 시각 뉴스)[^.\n]*',
    r'SNS\s*기사보내기',
    r'카카오톡로\s*기사보내기',
    r'URL복사로\s*기사보내기',
    r'다른\s*공유\s*찾기',
    r'기사와\s*상관\s*없는\s*자료사진',
    r'fullscreen',
    r'News1\s*DB',
    r'ENM\s*본\s*기사는\s*스포일러를\s*포함하고\s*있습니다.',
    r'photo',
    r'일러스트\s*NEWS\s*IMAGE',
    r'News1',
    r'기사의\s*본문\s*내용은\s*이\s*글자크기로\s*변경됩니다.',
    r'아래\s*텍스트는\s*속기초안이며,\s*추후\s*업데이트\s*됩니다.',
    r'아래\s*텍스트는\s*실제\s*방송\s*내용과\s*차이가\s*있을\s*수\s*있으니\s*보다\s*정확한\s*내용은\s*방송으로\s*확인하시기\s*바랍니다.',
    r'한강타임즈는\s*언제나\s*여러분의\s*제보를\s*기다립니다.',
    r'※.*|▶.*|★.*'
]

for pattern in patterns_to_remove:
    kr_news['text'] = kr_news['text'].apply(lambda x: re.sub(pattern, '', x))


kr_news['text'] = kr_news['text'].apply(lambda x: re.sub(r'[^A-Za-z0-9가-힇.,\"\'\:\·\!\?\-\%\~\&]', ' ', x))
# 공백 정리
kr_news['text'] = kr_news['text'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())
kr_news['text'] = kr_news['text'].apply(lambda x: re.sub(r'\.{2,}', '.', x))

In [27]:
#제목이 빈 문자열 행 제거
removed_empty_title = kr_news[kr_news['title'].astype(str).str.strip() == ''].copy()
removed_empty_title["reason"] = "제목 빈 문자열"
kr_news = kr_news[kr_news['title'].astype(str).str.strip() != '']

#본문이 20자 이하인 행 제거
removed_short_text = kr_news[kr_news['text'].astype(str).str.len() <= 20].copy()
removed_short_text["reason"] = "본문 20자 이하"
kr_news = kr_news[kr_news['text'].astype(str).str.len() > 20]

print("제목이 빈 문자열이라서 삭제된 행 : ",len(removed_empty_title))
print("본문이 20자 이하라서 작제된 행 : ",len(removed_short_text))

제목이 빈 문자열이라서 삭제된 행 :  0
본문이 20자 이하라서 작제된 행 :  0


In [28]:
#스포츠 종목 및 리그
SPORT_KEYWORDS = [
    "야구", "농구", "축구", "골프", "e스포츠", "LCK", "롤드컵",
    "KBO", "KBL", "KPGA", "프로야구", "프로농구"
]

#스포츠 뉴스에서 자주 사용되는 단어들
SPORT_CONTEXT_KEYWORDS = [
    "경기", "시즌", "우승", "패배", "완패", "승리", "라운드", "감독", 
    "리그", "순위", "스코어", "1위", "2위", "3위", "시드", "예선", "결승", 
    "챔피언", "대회", "출전", "득점", "홈런", "발로란트","공동", "GEN.G", 
    "코치", "포스트시즌", "정규시즌", "완파", "역전", "프로", "진출", "연봉"
]


def check_sports_reason(title, text):
    content = (title + " " + text).lower()

    found_sports = [k for k in SPORT_KEYWORDS if k.lower() in content]
    found_context = [k for k in SPORT_CONTEXT_KEYWORDS if k.lower() in content]
    has_business = any(k in content for k in ["매출", "실적", "계약", "공시", "출시", "사업"])

    # 스포츠 키워드 존재 + 문맥 키워드 3개 이상 + 비즈니스 문맥 아님
    if found_sports and len(found_context) >= 3 and not has_business:
        return True
    else:
        return False

sports_flags = kr_news.apply(lambda r: check_sports_reason(str(r['title']), str(r['text'])), axis=1)

removed_sports = kr_news[sports_flags].copy()
removed_sports["reason"] = "스포츠 뉴스"
kr_news = kr_news[~sports_flags]

print("스포츠 관련 기사 제거:", len(removed_sports))


스포츠 관련 기사 제거: 115


In [30]:
removed_all = pd.concat([
    removed_duplicates,
    removed_nan,
    removed_pattern,
    removed_mismatch,
    removed_empty_title,
    removed_short_text,
    removed_sports
], ignore_index=True)

In [31]:
output_path = 'trash_news.csv'

if os.path.exists(output_path):
    df_existing = pd.read_csv(output_path, encoding='utf-8-sig')
    
    print("기존 뉴스 개수 : ",len(df_existing))
    print("추가 뉴스 개수 : ",len(removed_all))
    # 새 데이터와 기존 데이터 병합
    removed_all = pd.concat([df_existing, removed_all], ignore_index=True)
    # 이거는 혹시 크롤링 id가 겹칠 수 있으면 지우는 걸로 하겠습니다.(지금은 테스트 중이라!!!!!!!!!!!!!!!!)
    removed_all = removed_all.drop_duplicates(subset=['crawled_news_id'], keep='last')
else:
    print("파일 새로 추가 중!!")
print("전체 뉴스 개수 : ",len(removed_all))

removed_all['pub_date'] = pd.to_datetime(removed_all['pub_date'], errors='coerce')

# 날짜순 정렬 (최신 데이터가 위로)
removed_all = removed_all.sort_values('pub_date', ascending=False).reset_index(drop=True)
removed_all.to_csv(output_path, index=False, encoding="utf-8-sig")
print("trash_news.csv로 저장 완료!!")

파일 새로 추가 중!!
전체 뉴스 개수 :  373
trash_news.csv로 저장 완료!!


In [32]:
output_path = 'news_1st.csv'

if os.path.exists(output_path):
    df_existing = pd.read_csv(output_path, encoding='utf-8-sig')
    
    print("기존 뉴스 개수 : ",len(df_existing))
    print("추가 뉴스 개수 : ",len(kr_news))
    # 새 데이터와 기존 데이터 병합
    kr_news = pd.concat([df_existing, kr_news], ignore_index=True)
    # 이거는 혹시 크롤링 id가 겹칠 수 있으면 지우는 걸로 하겠습니다.(지금은 테스트 중이라!!!!!!!!!!!!!!!!)
    kr_news = kr_news.drop_duplicates(subset=['crawled_news_id'], keep='last')
else:
    print("파일 새로 추가 중!!")
print("전체 뉴스 개수 : ",len(kr_news))

kr_news['pub_date'] = pd.to_datetime(kr_news['pub_date'], errors='coerce')

# 날짜순 정렬 (최신 데이터가 위로)
kr_news = kr_news.sort_values('pub_date', ascending=False).reset_index(drop=True)
kr_news.to_csv(output_path, index=False, encoding="utf-8-sig")
print("news_1st.csv로 저장 완료!!")

파일 새로 추가 중!!
전체 뉴스 개수 :  7543
news_1st.csv로 저장 완료!!
